<a href="https://colab.research.google.com/github/AndreManganelli/Chatbot-IA-KnowledgeBase/blob/main/chatbot_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import numpy as np
import json
import datetime
import random as random
import google.generativeai as genai

# Define a variável de ambiente para a API Key do Gemini
GOOGLE_API_KEY="INSERT API-KEY HERE - INSIRA SUA API KEY AQUI"
genai.configure(api_key=GOOGLE_API_KEY)


# Carrega o arquivo JSON com problemas e soluções
def carregar_base_conhecimento(nome_arquivo="base_conhecimento.json"):
  """Carrega a base de conhecimento a partir de um arquivo JSON."""
  with open(nome_arquivo, 'r', encoding='utf-8') as f:
    return json.load(f)


# Função para gerar um número de chamado
def gerar_numero_chamado():
  """Gera um número de chamado aleatório."""
  return random.randint(100000, 999999)

# Função para salvar o histórico de atendimentos
def salvar_historico(historico, nome_arquivo="historico_atendimentos.json"):
  """Salva o histórico de atendimentos em um arquivo JSON."""
  with open(nome_arquivo, 'w', encoding='utf-8') as f:
    json.dump(historico, f, indent=4)

# Função principal do chatbot
def chatbot():
  """Função principal do chatbot."""

  base_conhecimento = carregar_base_conhecimento()
  historico_atendimentos = []

  print("Olá! 👋 Bem-vindo ao nosso suporte técnico.")

  while True:
    print("\nDigite sua dúvida ou problema:")
    duvida = input("> ")

    # Procura a solução na base de conhecimento
    solucao_encontrada = False
    for problema in base_conhecimento:
      if problema["problema"].lower() in duvida.lower():
        print(f"Encontrei uma solução para você:\n{problema['solucao']}")
        solucao_encontrada = True
        break

    # Se não encontrou na base de conhecimento, utiliza o Gemini
    if not solucao_encontrada:
      # Utiliza o modelo Gemini para gerar uma resposta

      model = genai.GenerativeModel("gemini-1.0-pro")

      result = model.generate_content([duvida])

      print(f"Resposta do técnico:\n{result.text}")

    # Pergunta se há mais dúvidas
    mais_duvidas = input("\nVocê tem mais alguma dúvida? (sim/não): ").lower()
    if mais_duvidas != "sim":
      # Gera o número do chamado e solicita a nota de atendimento
      numero_chamado = gerar_numero_chamado()
      print(f"\nNúmero do seu chamado: {numero_chamado}")

      while True:
        try:
          nota = int(input("Por favor, avalie nosso atendimento de 1 a 5 (sendo 5 a nota máxima): "))
          if 1 <= nota <= 5:
            break
          else:
            print("Nota inválida. Digite um número entre 1 e 5.")
        except ValueError:
          print("Entrada inválida. Digite um número entre 1 e 5.")

      # Armazena a interação no histórico
      historico_atendimentos.append({
        "data_hora": datetime.datetime.now().isoformat(),
        "numero_chamado": numero_chamado,
        "nota": nota
      })
      salvar_historico(historico_atendimentos)
      print("Obrigado pelo seu feedback! 😊")
      break

# Comando para visualizar o histórico de atendimentos
def visualizar_historico():
  """Carrega e exibe o histórico de atendimentos."""
  try:
    historico = json.load(open("historico_atendimentos.json", 'r', encoding='utf-8'))
    if historico:
      print("\nHistórico de Atendimentos:")
      for atendimento in historico:
        print(f"- Data e Hora: {atendimento['data_hora']}")
        print(f"  Número do Chamado: {atendimento['numero_chamado']}")
        print(f"  Nota: {atendimento['nota']}\n")
    else:
      print("Nenhum atendimento registrado no histórico.")
  except FileNotFoundError:
    print("Arquivo de histórico não encontrado.")

# Inicia o chatbot
if __name__ == "__main__":
  chatbot()

  # Loop para comandos adicionais
  while True:
    comando = input("\nDigite um comando (historico ou sair): ").lower()
    if comando == "historico":
      visualizar_historico()
    elif comando == "sair":
      break
    else:
      print("Comando inválido.")